This summarizer users GloVe embeddings. The necessary file glove.GB.100d.txt can be downloaded from the internet GloVe website https://nlp.stanford.edu/projects/glove/.


I have a text document which is "combined.txt" This is 10 text documents from mlg.ucd.ie/datasets/bbc.html. They correspond to news articles, in this instance all about football.

This summarizer uses pagerank to rank the sentences.

There are numerous methods for text summarization. This works for smaller documents.

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from nltk.tokenize import sent_tokenize


In [ ]:

#df = pd.read_csv('reviews.csv')
#print(df.head())


file=open("combined.txt", 'r')
file_contents=file.read()
print(file_contents)

sentences=[]
for s in file_contents:
    sentences.append(s)
    
stop_words = stopwords.words('english')


def remove_stopwords(sentences):                                     
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new   #sen_new

def process_sents(sentences):
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    return clean_sentences


word_embeddings = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

def summarize_sents(sentences):
    sentence_vectors = []
    for i in sentences:
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
      sentence_vectors.append(v)

    sim_mat = np.zeros([len(sentences), len(sentences)])

    for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

print("summary")
print("")

sn =10
for i in range(sn):
    print(ranked_sentences[i][1])

for i in range(10):
    print()
    sentences = sent_tokenize(df['review'][i])
    summarize_sents(process_sents(sentences))
